1. Скачайте датасет MNIST.
2. Изучите выборку (какие там картинки, сколько картинок в каждом классе).
3. Реализуйте загрузку датасета в память с диска с помощью OpenCV. Использовать готовые реализации датасетов и даталоудеров торча запрещено. Пока можно не заморачиваться многопоточностью.
4. Реализуйте аугментацию данных: поворот картинки на определенное кол-во градусов (задается аргументом), зашумление картинки. Реализовывать на numpy. Использовать готовые решения тоже запрещено.
5. Реализуйте способ скормить это нейросети (нормализация, приведение к одному разрешению).
6. Напишите код нейросети. Использовать Sequential запрещено. Модель пишете руками, наследуясь от torch.nn.Module.
7. Реализуйте цикл обучения нейросети. Использовать готовые трейнеры и model.fit() запрещено. Кол-во эпох, learning rate и прочие параметры задаются аргументами.
8. Напишите код для инференса и оценки качества работы нейронной сети.
9. Оценка качества должна заключаться в подсчете Accuracy, Recall, Precision, F-мера. Пользоваться готовыми решениями тоже запрещено.
10. Оформите это как удобную библиотечку, которую можно склонить с гитхаба, положить куда надо файлы весов модели, выборку и обучить / инференсить модель.

In [ ]:
import os
import urllib.request
import gzip
import numpy as np
import cv2
import torch
import torch.nn as nn

In [ ]:
path = "./MNIST/archive/"
os.makedirs(path, exist_ok = True)

source = {"training_images" : "http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz",
          "training_labels" : "http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz",
          "test_images" : "http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz",
          "test_labels" : "http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"
        }

In [ ]:
data={}

for key in source:
  data[key] = source[key].split('/')[-1]
  if os.path.exists(path + data[key]):
    print(data[key] + " was here long before you were born...")
  else:
    urllib.request.urlretrieve(source[key], path + data[key])
    print(data[key] + " downloaded right now!")

In [ ]:
dataset = {}
#path = "/MNIST/archive"

# Images
for key in ("training_images", "test_images"):
  with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
    dataset[key] = np.frombuffer(mnist.read(), 'B', offset = 16).reshape(-1, 28 * 28)

# Labels
for key in ("training_labels", "test_labels"):
    with gzip.open(os.path.join(path, data[key]), "rb") as mnist:
      dataset[key] = np.frombuffer(mnist.read(), np.uint8, offset = 8)

x_train, y_train, x_test, y_test = (
    dataset["training_images"],
    dataset["training_labels"],
    dataset["test_images"],
    dataset["test_labels"])

In [ ]:
#cv2 check here

#normalize
training_sample, test_sample = 10000, 10000
training_images = x_train[0:training_sample] / 255
test_images = x_test[0:test_sample] / 255

#toDimensions
def trick(labels, dimension=10):
    trick = labels[..., None] == np.arange(dimension)[None]
    return trick.astype(np.float64)

training_labels = trick(y_train[:training_sample])
test_labels = trick(y_test[:test_sample])

print(training_labels[0])
print(training_labels[1])
print(training_labels[2])

print(y_train[0])
print(y_train[1])
print(y_train[2])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
5
0
4


In [ ]:
seed = 123456
gen = np.random.default_rng(seed)

learning_rate = 0.005
epochs = 100

weights_1 = 0.15 * gen.random((784, 256)) - 0.01
weights_2 = 0.15 * gen.random((256, 10)) - 0.01

training_loss = []
training_accuracy = []
test_loss = []
test_accuracy = []

def reLu(x):
    return (x >= 0) * x

def reLuDiriv(x):
    return x >= 0

for j in range(epochs):
  loss = 0.0
  accuracy = 0

  for i in range(len(training_images)):
        layer_0 = training_images[i]
        layer_1 = np.dot(layer_0, weights_1)
        layer_1 = reLu(layer_1)
        #dropout_mask = gen.integers(0, 3, layer_1.shape)
        #layer_1 *= dropout_mask * 3
        layer_2 = np.dot(layer_1, weights_2)
        #TODO: softmax func

        loss += np.sum((training_labels[i] - layer_2) ** 2)
        accuracy += int(np.argmax(layer_2) == np.argmax(training_labels[i]))
        #layer_1_delta *= dropout_mask

  training_loss.append(loss)
  training_accuracy.append(accuracy)

  result = reLu(test_images @ weights_1) @ weights_2

  tloss = np.sum((test_labels - result) ** 2)
  taccuracy = np.sum(np.argmax(result, 1) == np.argmax(test_labels, 1))

  test_loss.append(tloss)
  test_accuracy.append(taccuracy)

  print(( f"Epoch: {j}\n"
          f"  Training set loss: {loss / len(training_images):.3f}\n"
          f"  Training set accuracy: {accuracy / len(training_images)}\n"
          f"  Test set loss: {tloss / len(test_images):.3f}\n"
          f"  Test set accuracy: {taccuracy / len(test_images)}"))

NameError: ignored